In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np

In [2]:
db_name = "V:\\Databases\\reporting.db"
conn = sqlite3.connect(db_name)

In [30]:
inpatient_snf = pd.read_sql('SELECT * FROM inpatient_snf', conn)

In [64]:
snf_reasons = inpatient_snf[inpatient_snf['admit_reason'].notnull()]['admit_reason'].copy()
skilled = snf_reasons[snf_reasons.str.contains('skilled', case=False)].unique()
custodial = snf_reasons[snf_reasons.str.contains('custodial|EOL|end of life|long term', case=False)].unique()
respite = snf_reasons[snf_reasons.str.contains('respite', case=False)].unique()
other = [reason for reason in snf_reasons.unique() if reason not in skilled.tolist() + custodial.tolist() + respite.tolist()]

In [65]:
other

['Pt/Ot, Strengthening, Fx Care', 'Skiled Stay', 'Wound Care, Rehab', 'Other']

In [48]:
skilled.unique()

array(['Skilled'], dtype=object)

In [66]:
{x:'skilled' for x in skilled}

{'Skilled': 'skilled'}

In [155]:
utl_dict = {}
utl_dict['er_non'] = pd.read_csv('er_non.csv')
utl_dict['er_adm'] = pd.read_csv('er_adm.csv')
utl_dict['inpatient'] = pd.read_csv('inpatient.csv')
utl_dict['ut_grid_er'] = pd.read_csv('ut_grid_er.csv')
utl_dict['ut_grid_inp'] = pd.read_csv('ut_grid_inp.csv')

tables = {}
tables['enrollment'] = pd.read_csv('enrollment.csv')
tables['enrollment'].rename(columns={'MemberID': 'member_id', 'EnrollmentDate': 'enrollment_date'}, inplace=True)

In [156]:
def code_y_n(df):
    """
    Takes a pandas dataframe and codes any columns containing Yes/No as 1/0
    """
    for col in df.columns:
        if 'No' in df[col].unique():
            df[col] = df[col].str.title()
            df[col].replace({'Yes':1, 'No':0}, inplace=True)
            

def discharge_admit_diff(df, sql_table='', update=False, admit_diff=False, admit_type=''):
    dff = df.copy()
    if update:
        if admit_type != '':
            admission_sql = f"AND admission_type == '{admit_type}'"
        else:
            admission_sql = ''
        conn = sqlite3.connect(db_name)

        q = f"""SELECT {','.join([col for col in df.columns if col != 'merge'])} FROM {sql_table}
        WHERE member_id IN {tuple(df.member_id.unique())}
        {admission_sql};"""

        current_table = pd.read_sql(q, conn, parse_dates = [col for col in df.columns if 'date' in col])

        df = current_table.append(df, sort=False)
        df.reset_index(inplace=True)
        
    if admit_diff:
        diff_date = 'admission_date'
        sorted_df = df.sort_values(['member_id', 'admission_date'], 
                               ascending=False).reset_index(drop=True).copy()
    else:
        diff_date = 'discharge_date'
        sorted_df = df.sort_values(['member_id', 'admission_date', 'discharge_date'], 
                               ascending=False).reset_index(drop=True).copy()
    #sort dataframe by member_id and then admission date
    
    
    sorted_df['days_since_last_admission'] = np.nan

    #iterate through unique member_ids
    for mem_id in sorted_df.member_id.unique():
        #if the member_id appears more than once in the df
        if sorted_df[sorted_df.member_id == mem_id].shape[0] > 1:
            #iterate through each occurrence, first occurrence
            #will be the most recent admission_date
            for i in sorted_df[sorted_df.member_id == mem_id].index[:-1]:
                #find difference between current admission_date and
                #most recent discharge_date
                sorted_df.at[i, 'days_since_last_admission'] = (sorted_df.at[i, 'admission_date'] - sorted_df.at[(i+1), diff_date]) / np.timedelta64(1, 'D')

    sorted_df.reset_index(drop=True, inplace=True)
    if update:
        dff = dff.merge(sorted_df[['member_id', 'admission_date', 'days_since_last_admission']],
                        on=['member_id', 'admission_date'], how='left')
        return dff
    
    return sorted_df

In [157]:
inpatient_rename_dict = {
    "MemberID": "member_id",
    "AdmissionDate": "admission_date",
    "DischargeDate": "discharge_date",
    "LOSDays": "los",
    "FacilityName": "facility",
    "DischargeDisposition": "discharge_reason",
    "AdmitReason": "admit_reason",
    "AdmissionType": "admission_type",
}
# cols to drop from inpatient Cognify dataframe
inpatient_drop = [
    "textBox5",
    "Center",
    "ParticipantName",
    "PCP",
    "ICUDays",
    "Diagnosis",
    "Readmit",
    "AdmissionScheduled",
]
# names of er Cognify cols
er_rename_dict = {
    "MemberID": "member_id",
    "AdmissionDate": "admission_date",
    "Facility": "facility",
}
# cols to drop from er Cognify dataframe
er_drop_cols = ["textBox5", "textBox2", "Center", "ParticipantName", "PCP", "Diagnosis"]
# cols not needed in UR Grid
utl_drop_cols = [
    "Ppt Name",
    "Member ID",
    "Date of visit",
    "Date of Discharge",
    "# of days hospitalized",
    "Last MD visit",
    "Last RN visit",
    "Hospital",
    "CMS Preventable",
    "if yes, what was done",
    "Possible interventions that could have prevented visit",
    "Additional interventions 1",
    "Additional interventions 2",
    "Additional interventions 3",
    "Comments",
    "End of life",
]
utl_rename_dict = {
    "# of days prior to hospital visit": "days_MD",
    "# of days prior to hospital visit.1": "days_RN",
    "Visit type": "visit_type",
    "Time of visit": "time",
    "Lives": "living_situation",
    "Adm from": "admitted_from",
    "Facilty adm from": "admitting_facility",
    "Reason for visit": "reason",
    "Discharge Diagnosis": "discharge_dx",
    "Related to": "related",
    "Sent by On-call staff": "sent_by_oc",
    "PACE staff aware of S/S prior": "aware_ss",
    "PACE aware of visit prior": "aware_visit",
    "Preventable/Avoidable": "preventable",
}

In [158]:
def clean_utlization(utl_dict, inpatient_cols, inpatient_drop, er_cols, er_drop, ut_cols, utl_drop_cols, update=False):
    enrollment_new = tables['enrollment'][['member_id', 'enrollment_date']]
    enrollment = enrollment_new.copy()
    if update:
        enrollment_db = pd.read_sql("SELECT member_id, enrollment_date FROM enrollment", conn, parse_dates = ['enrollment_date'])
        enrollment = enrollment_db.append(enrollment_new)
        
    #drop and rename cols for Cognify reports
    utl_dict['inpatient'].drop(inpatient_drop, axis=1, inplace=True)
    utl_dict['inpatient'].rename(columns=inpatient_cols, inplace=True)

    utl_dict['er_adm'].drop(er_drop, axis=1, inplace=True)
    utl_dict['er_adm'].rename(columns=er_cols, inplace=True)

    utl_dict['er_non'].drop(er_drop, axis=1, inplace=True)
    utl_dict['er_non'].rename(columns=er_cols, inplace=True)

    cognify_faciliity_fix = {'Our Lady of Fatima Hospital': 'Fatima Hospital',
                            'The Miriam Hospital' : 'Miriam Hospital',
                            'Westerly Hospital ': 'Westerly Hospital',
                            'Roger Williams Hospital': 'Roger Williams Medical Center',
                            'Roger Williams Cancer Center': 'Roger Williams Medical Center',
                            'St. Elizabeth Manor': 'St. Elizabeth Home'}
    
    #replace some Cognify report facility quirks
    #can these be changed when being input?
    utl_dict['inpatient']['facility'].replace(cognify_faciliity_fix,
                                          inplace=True)
    utl_dict['er_adm']['facility'].replace(cognify_faciliity_fix,
                                          inplace=True)
    utl_dict['er_non']['facility'].replace(cognify_faciliity_fix,
                                          inplace=True)

    utl_dict['inpatient']['merge'] = (utl_dict['inpatient']['member_id'].astype(str) +
                                  utl_dict['inpatient']['admission_date'].astype(str) +
                                  utl_dict['inpatient']['facility'])

    utl_dict['er_adm']['merge'] = (utl_dict['er_adm']['member_id'].astype(str) +
                                  utl_dict['er_adm']['admission_date'].astype(str) +
                                  utl_dict['er_adm']['facility'])

    utl_dict['er_non']['merge'] = (utl_dict['er_non']['member_id'].astype(str) +
                                  utl_dict['er_non']['admission_date'].astype(str) +
                                  utl_dict['er_non']['facility'])

    #Cognify inpatetient does not indicate ER or not
    #check if inpatient stay is in the ER admited report
    utl_dict['inpatient']['er'] = np.where(utl_dict['inpatient']['merge'].isin(utl_dict['er_adm']['merge'].tolist()),
                                                       1, 0)

    #change some hospital names in the manual utl grids
    utl_hospital = {'Kent' : 'Kent Hospital',
                'kent': 'Kent Hospital',
                'RIH' : 'Rhode Island Hospital', 
                'Landmark' : 'Landmark Medical Center', 
                'Miriam' : 'Miriam Hospital',
                'RWMC' : 'Roger Williams Medical Center',
                'Butler' : 'Butler Hospital',
                'SCH' : 'South County Hospital',
                'Fatima' : 'Fatima Hospital',
                'FirstHealth*' : 'FirstHealth Moore Reginal Hospital',
                'East Side urgent care' : 'East Side Urgent Care',
                'Westerly': 'Westerly Hospital',
                'SCC *': 'Carolinas Hosptial System',
                'W&I' : 'Women & Infants Hospital'}

    utl_dict['ut_grid_inp']['Hospital'].replace(utl_hospital, inplace=True)

    utl_dict['ut_grid_er']['Hospital'].replace(utl_hospital, inplace=True)

    #converting the xls file to csv if giving us some ghost rows
    utl_dict['ut_grid_inp'].dropna(subset=['Member ID'], inplace=True)
    utl_dict['ut_grid_er'].dropna(subset=['Member ID'], inplace=True)

    utl_dict['ut_grid_inp']['Member ID'] = utl_dict['ut_grid_inp']['Member ID'].astype(int)
    utl_dict['ut_grid_er']['Member ID'] = utl_dict['ut_grid_er']['Member ID'].astype(int)

    utl_dict['ut_grid_inp']['merge'] = (utl_dict['ut_grid_inp']['Member ID'].astype(str) +
                                  utl_dict['ut_grid_inp']['Date of visit'].astype(str) +
                                  utl_dict['ut_grid_inp']['Hospital'])

    #utl_dict['ut_grid_inp']['merge'] = utl_dict['ut_grid_inp']['merge'].str.replace('.', '')

    utl_dict['ut_grid_er']['merge'] = (utl_dict['ut_grid_er']['Member ID'].astype(str) +
                                  utl_dict['ut_grid_er']['Date of visit'].astype(str) +
                                  utl_dict['ut_grid_er']['Hospital'])
    #utl_dict['ut_grid_er']['merge'] = utl_dict['ut_grid_inp']['merge'].str.replace('.', '')
                                  
    #check that all UR Grid visits are in Cognify correctly
    grid_vs_cognify = utl_dict['ut_grid_inp'][(-utl_dict['ut_grid_inp']['merge'].isin(utl_dict['inpatient']['merge'].tolist()) &
                        utl_dict['ut_grid_inp']['Hospital'].notnull())][['Member ID',
                                                                          'Ppt Name', 
                                                                          'Date of visit', 
                                                                          'Date of Discharge',
                                                                          'Hospital', 'merge']]
    
    if not grid_vs_cognify.empty:
        grid_vs_cognify.to_csv('ut_grid_cognify_diffs.csv', index=False)
        utl_dict['inpatient'][utl_dict['inpatient']['member_id'].isin(grid_vs_cognify['Member ID'])].to_csv('inp_for_diff.csv', index=False)
        raise AssertionError('All UR Grid Inpatient Visits are not in Cognify correctly.')

    #check that all UR Grid ER visits are either in the Cognify Inpatient Report
    #or in the ER Non Admit Report
    #this is where there are Cognify inpatient stays in the UT Grid ER Tab
    er_inp_check = utl_dict['inpatient'][(utl_dict['inpatient']['merge'].isin(utl_dict['ut_grid_er']['merge'].tolist()))]['merge']

    #this is all UT Grid ER Visits that are not in the Congify ER Only List
    missing_cog = utl_dict['ut_grid_er'][(-(utl_dict['ut_grid_er']['merge'].isin(utl_dict['er_non']['merge'].tolist())) &
                                      utl_dict['ut_grid_er']['Hospital'].notnull())][['Member ID', 'Ppt Name',
                                                                                       'Date of visit', 'Date of Discharge',
                                                                                       'Hospital', 'merge']]
    #this is any UR Grid ER Visits that is not in either Cognify report
    missing_ur_er_visits = missing_cog[-(missing_cog['merge'].isin(er_inp_check))]
    
    if not missing_ur_er_visits.empty:
        missing_ur_er_visits.to_csv('ut_grid_cognify_diffs_er.csv', index=False)
        raise AssertionError('All UR Grid ER Visits are not in Cognify correctly.')

    #drop cols
    utl_dict['ut_grid_inp'].drop(utl_drop_cols, axis=1, inplace=True)
    utl_dict['ut_grid_er'].drop(utl_drop_cols, axis=1, inplace=True)

    #rename cols
    utl_dict['ut_grid_inp'].rename(columns=utl_rename_dict, inplace=True)
    utl_dict['ut_grid_er'].rename(columns=utl_rename_dict, inplace=True)

    #replace weird excel blank values with np.nan
    utl_dict['ut_grid_inp']['days_MD'] = np.where(utl_dict['ut_grid_inp']['days_MD'].astype(float)>=1000, np.nan, utl_dict['ut_grid_inp']['days_MD'])
    utl_dict['ut_grid_inp']['days_RN'] = np.where(utl_dict['ut_grid_inp']['days_RN'].astype(float)>=1000, np.nan, utl_dict['ut_grid_inp']['days_RN'])

    utl_dict['ut_grid_er']['days_MD'] = np.where(utl_dict['ut_grid_er']['days_MD'].astype(float)>=1000, np.nan, utl_dict['ut_grid_er']['days_MD'])
    utl_dict['ut_grid_er']['days_RN'] = np.where(utl_dict['ut_grid_er']['days_RN'].astype(float)>=1000, np.nan, utl_dict['ut_grid_er']['days_RN'])

    #drop any scheduled visits, data is not tracked for these
    drop_sched = utl_dict['ut_grid_inp'][utl_dict['ut_grid_inp']['visit_type'] == 'Scheduled'].index.tolist()
    utl_dict['ut_grid_inp'].drop(drop_sched, inplace=True)

    utl_dict['ut_grid_inp'].drop('visit_type', axis=1, inplace=True)

    #create OBS column
    utl_dict['ut_grid_er']['visit_type'] = np.where(utl_dict['ut_grid_er']['visit_type'] == 'OBS', 1, 0)
    utl_dict['ut_grid_er'].rename(columns={'visit_type':'observation'}, inplace=True)

    inp_obs = utl_dict['ut_grid_er'][(utl_dict['ut_grid_er']['merge'].isin(er_inp_check)) & 
                       (utl_dict['ut_grid_er']['observation'] == 1)]['merge'].tolist()

    utl_dict['inpatient']['observation'] = np.where(utl_dict['inpatient']['merge'].isin(inp_obs), 1, 0)

    #removal of UR Grid ER Visits that are not ER Only but instead
    #are ER to inpatient
    utl_dict['ut_grid_er'] = utl_dict['ut_grid_er'][-utl_dict['ut_grid_er']['merge'].isin(er_inp_check)]

    #Seperate out inpatient Hospital and Inpatient Psych Visits
    #this is so days since last visit refers to a visit of a certain type

    hosp_list = ['Rhode Island Hospital', 'Landmark Medical Center', 
                'Kent Hospital', 'Roger Williams Hospital', 'Fatima Hospital',
                'Miriam Hospital', 'Westerly Hospital', 'Women & Infants Hospital']

    fill_admission_type_na = ['Acute Hospital' if facility in hosp_list else np.nan for facility in utl_dict['inpatient']['facility']]

    utl_dict['inpatient']['admission_type'] = np.where(utl_dict['inpatient']['admission_type'].isnull(),
                                                        fill_admission_type_na,
                                                        utl_dict['inpatient']['admission_type'])

    inpatient_hosp = utl_dict['inpatient'][utl_dict['inpatient'].admission_type.isin(['Acute Hospital',
                                                                                  np.nan])].copy().reset_index(drop=True)

    inpatient_pysch = utl_dict['inpatient'][utl_dict['inpatient'].admission_type.isin(['Psych Unit / Facility',
                                                                                  np.nan])].copy().reset_index(drop=True)

    #Create datetime Cols
    inpatient_hosp['admission_date'] = pd.to_datetime(inpatient_hosp['admission_date'])
    inpatient_hosp['discharge_date'] = pd.to_datetime(inpatient_hosp['discharge_date'])

    inpatient_pysch['admission_date'] = pd.to_datetime(inpatient_pysch['admission_date'])
    inpatient_pysch['discharge_date'] = pd.to_datetime(inpatient_pysch['discharge_date'])

    inpatient_hosp.drop_duplicates(inplace=True)
    inpatient_pysch.drop_duplicates(inplace=True)

    #Calculate Day Since Last Admit
    inpatient_hosp = discharge_admit_diff(inpatient_hosp, sql_table='inpatient', admit_type='Acute Hospital', update=update)
    inpatient_pysch = discharge_admit_diff(inpatient_pysch, sql_table='inpatient', admit_type='Psych Unit / Facility', update=update)

    #these are all weird copies that are checking the Admission Date against the same stays Discharge Date
    inpatient_hosp['days_since_last_admission'] = np.where(inpatient_hosp['days_since_last_admission'] <= 0, np.nan,
                                                           inpatient_hosp['days_since_last_admission'])

    inpatient_pysch['days_since_last_admission'] = np.where(inpatient_pysch['days_since_last_admission'] <= 0, np.nan,
                                                            inpatient_pysch['days_since_last_admission'])
       
    #put them back together
    inpatient = inpatient_hosp.append(inpatient_pysch)

    #merge inpatient Cognify report with the UR Grid Inpatient
    inpatient = inpatient.merge(utl_dict['ut_grid_inp'], on='merge', how='left').drop_duplicates()
  
    #merge with enrollment to create a column indicating
    #if the visit was within 6 months of enrollment

    inpatient = inpatient.merge(enrollment[['member_id','enrollment_date']])

    inpatient['enrollment_date'] = pd.to_datetime(inpatient['enrollment_date'])
    inpatient['admission_date'] = pd.to_datetime(inpatient['admission_date'])

    inpatient['w_six_months'] = np.where((inpatient['admission_date'].dt.to_period('M') -
                                           inpatient['enrollment_date'].dt.to_period('M')).apply(lambda x: x.freqstr[:-1]).replace('',
                                                                                                                                       np.nan).astype(float) <= 6,
                                          1, 0)

    #Create day of the week column
    inpatient['dow'] = inpatient['admission_date'].dt.weekday
    inpatient['dow'].replace({0:'Monday', 1:'Tuesday', 2:'Wednesday',
    3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}, inplace=True)
    
    #Code any Yes/No cols as 1/0
    code_y_n(inpatient)

    #Drop left over enrollment data and the merge column - we are done with them
    inpatient.drop(['enrollment_date', 'merge'], axis=1, inplace=True)

    #create datetime cols
    inpatient['admission_date'] = pd.to_datetime(inpatient['admission_date'])
    inpatient['discharge_date'] = pd.to_datetime(inpatient['discharge_date'])

    inpatient.drop_duplicates(subset = ['member_id', 'admission_date', 'facility'], inplace=True)

    for col in inpatient.columns:
        if inpatient[col].dtype == 'O':
            inpatient[col] = inpatient[col].str.title()

    #Merge Cognify ER Only and UR Grid ER reports
    er_only = utl_dict['er_non'].merge(utl_dict['ut_grid_er'], on='merge', how='left').drop_duplicates()

    er_only = er_only.merge(enrollment[['member_id','enrollment_date']])
    
    #merge with enrollment to create a column indicating
    #if the visit was within 6 months of enrollment
    er_only['enrollment_date'] = pd.to_datetime(er_only['enrollment_date'])
    er_only['admission_date'] = pd.to_datetime(er_only['admission_date'])

    er_only['w_six_months'] = np.where((er_only['admission_date'].dt.to_period('M') - 
                                    er_only['enrollment_date'].dt.to_period('M')).apply(lambda x: x.freqstr[:-1]).replace('',
                                                                                                                         np.nan).astype(float) <= 6,
                                          1, 0)
    
    #Create day of the week column
    er_only['dow'] = er_only['admission_date'].dt.weekday
    er_only['dow'].replace({0:'Monday', 1:'Tuesday', 2:'Wednesday',
    3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}, inplace=True)
    
    #Code any Yes/No cols as 1/0
    code_y_n(er_only)

    #Drop left over enrollment data and the merge column - we are done with that
    er_only.drop(['enrollment_date', 'merge'], axis=1, inplace=True)

    #Create datetime cols
    er_only['admission_date'] = pd.to_datetime(er_only['admission_date'])

    er_only.drop_duplicates(inplace=True)

    #find days between visits for participants
    er_only = discharge_admit_diff(er_only, sql_table='er_only', admit_diff=True, update=update)

    #these are all weird copies that are checking the Admission Date against the same stays Discharge Date
    er_only['days_since_last_admission'] = np.where(er_only['days_since_last_admission'] <= 0, np.nan,
                                                    er_only['days_since_last_admission'])

    for col in er_only.columns:
        if er_only[col].dtype == 'O':
            er_only[col] = er_only[col].str.title()


    er_only.drop_duplicates(subset = ['member_id', 'admission_date', 'facility'], inplace=True)

    #start on Nursing Facility table
    inpatient_snf = utl_dict['inpatient'][utl_dict['inpatient']['admission_type'].isin(['Nursing Home',
                                                                                    'Rehab Unit / Facility',
                                                                                    'End of Life'])].copy()
    #replace long admit reasons with the core reason custodial/respite/skilled/other
    cust = {x:'custodial' for x in ['custodial placement', 'custodial stay', 'Custodial placement',
                         'Custodial', 'resumed custodial care', 'custodial palcement',
                         'EOL', 'custodial stay for EOL care', 'end of life', 'Custodial',
                         'custodial stay for colonscopy prep', 'eol care', 'need for long term care']}

    res = {x:'respite' for x in ['respite stay', 're', 'emergent respite stay', 'Respite', 'emergent respite']}

    oth = {x:'other' for x in ['behavioral concerns/unable to be managed at home', 'functional decline']}

    skill = {x:'skilled' for x in ['skilled services', 'sk', 'skilled stay', 'Skilled', 'weakness/PT/OT',
                       'skileld services', 'skilled','rehab', 'Rehab and Nursing Services',
                       'need for skilled wound care and pain management', 'Wound care', 'rehab']}

    reason_dict = {**cust, **res, **oth, **skill}

    inpatient_snf['admit_reason'].replace(reason_dict, inplace=True)

    #calculate LOS
    inpatient_snf['los'] = np.where(inpatient_snf.discharge_date.isnull(), np.nan, inpatient_snf.los)

    #create datetime cols
    inpatient_snf['admission_date'] = pd.to_datetime(inpatient_snf['admission_date'])
    inpatient_snf['discharge_date'] = pd.to_datetime(inpatient_snf['discharge_date'])

    #merge with enrollment to create a column indicating
    #if the visit was within 6 months of enrollment
    inpatient_snf = inpatient_snf.merge(enrollment[['member_id',
                                                          'enrollment_date']])

    inpatient_snf['enrollment_date'] = pd.to_datetime(inpatient_snf['enrollment_date'])

    inpatient_snf['w_six_months'] = np.where((inpatient_snf['admission_date'].dt.to_period('M') - 
                                          inpatient_snf['enrollment_date'].dt.to_period('M')).apply(lambda x: x.freqstr[:-1]).replace('',
                                                                                                                                     np.nan).astype(float) <= 6,
                                          1, 0)

    inpatient_snf.drop(['enrollment_date', 'er', 'merge'], axis=1, inplace=True)

    inpatient_snf.drop_duplicates(inplace=True)

    inpatient_snf = discharge_admit_diff(inpatient_snf,  sql_table='inpatient_snf', update=update)

    #these are all weird copies that are checking the Admission Date against the same stays Discharge Date
    inpatient_snf['days_since_last_admission'] = np.where(inpatient_snf['days_since_last_admission'] <= 0, np.nan,
                                                          inpatient_snf['days_since_last_admission'])

    inpatient_snf.drop_duplicates(subset = ['member_id', 'admission_date', 'facility'], inplace=True)

    for col in inpatient_snf.columns:
        if inpatient_snf[col].dtype == 'O':
            inpatient_snf[col] = inpatient_snf[col].str.title()

    return inpatient, er_only, inpatient_snf

In [160]:
tables['inpatient'], tables['er_only'], tables['inpatient_snf'] = clean_utlization(utl_dict,
                                                                                    inpatient_rename_dict, inpatient_drop,
                                                                                    er_rename_dict, er_drop_cols, utl_rename_dict,
                                                                                    utl_drop_cols, update=True)

C:\Users\snelson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


    member_id admission_date                       facility
0      100958       1/1/2019          Rhode Island Hospital
1     2000869       1/5/2019  Roger Williams Medical Center
2     2000912       1/7/2019          Rhode Island Hospital
3     2000906      1/17/2019  Roger Williams Medical Center
4      100261      1/19/2019  Roger Williams Medical Center
5     2001012      1/19/2019        Landmark Medical Center
6     2001020      1/21/2019                Miriam Hospital
7      100701      1/26/2019  Roger Williams Medical Center
8      100701      1/30/2019  Roger Williams Medical Center
9      100263       2/3/2019  Roger Williams Medical Center
10    1000851       2/4/2019          South County Hospital
11     100614       2/6/2019  Roger Williams Medical Center
12     100814      2/18/2019  Roger Williams Medical Center
13     100814      2/19/2019          Rhode Island Hospital
14    1000824      2/19/2019                Fatima Hospital
15    2001025      2/20/2019  Roger Will

In [79]:
pk = ['member_id', 'admission_date', 'facility']
table = 'er_only'

In [80]:
def update_sql_table(df, table_name, conn, primary_key):
    c = conn.cursor()

    #create temp table with possibly new data from cognify
    df.to_sql('temp', conn, index=False, if_exists='replace')

    #filters sql table for non new rows
    #and updates the cols
    filter_sql = f"""WHERE {primary_key[0]} = {table_name}.{primary_key[0]}"""

    try:
        for col in primary_key[1:]:
            filter_sql += f"""
                AND {col} = {table_name}.{col}
                """
    except IndexError:
        pass


    set_cols = [df_col for df_col in df.columns if df_col not in primary_key]

    set_sql = ', '.join([f"""{col} = (SELECT {col} FROM temp {filter_sql})""" for col in set_cols])

    exists_sql = f"""(SELECT {', '.join(set_cols)} FROM temp {filter_sql})"""

    c.execute(
        f"""
        UPDATE {table_name}
        SET {set_sql}
        WHERE EXISTS {exists_sql};
        """
    )
    conn.commit()

    #inserts new data if there is a primary key in the pandas df
    #that is not in the sql table
    insert_cols = ', '.join(col for col in df.columns)

    compare_pk_sql = ' AND '.join([ f"""f.{col} = t.{col}""" for col in primary_key])

    c.execute(
        f"""
        INSERT INTO {table_name} ({insert_cols})
        SELECT {insert_cols} FROM temp t
        WHERE NOT EXISTS 
            (SELECT {insert_cols} from {table_name} f
            WHERE {compare_pk_sql});
        """    
    )
    conn.commit()

In [162]:
update_sql_table(tables['er_only'], 'er_only', conn, pk)

In [46]:
conn = sqlite3.connect('V:\\Databases\\PaceDashboard.db')

In [47]:
conn_old = sqlite3.connect('V:\\Databases\\PaceDashboard_2019-04-17.db')

In [22]:
c = conn.cursor()

In [23]:
tables = [tup[0] for tup in c.execute('SELECT name FROM sqlite_master WHERE type = "table"').fetchall()]

In [33]:
table_diff = {}
for table in tables:
    q = f"SELECT * FROM {table}"
    
    old = pd.read_sql(q, conn_old)
    current = pd.read_sql(q, conn)
    
    table_diff[table] = pd.concat([old,current]).drop_duplicates(keep=False)

In [42]:
tables

['ppts',
 'addresses',
 'center_days',
 'demographics',
 'dx',
 'enrollment',
 'grievances',
 'inpatient',
 'er_only',
 'inpatient_snf',
 'burns',
 'falls',
 'infections',
 'med_errors',
 'wounds',
 'influ',
 'pneumo']

In [61]:
conn.close()
conn_old.close()

### Fix some stuff

In [ ]:
def clean_utlization(utl_dict, inpatient_cols, inpatient_drop, er_cols, er_drop, ut_cols, utl_drop_cols, update=False):
    enrollment_new = tables['enrollment'][['member_id', 'enrollment_date']]
    enrollment = enrollment_new.copy()
    
    if update:
        conn = sqlite3.connect(db_name)
        enrollment_db = pd.read_sql("SELECT member_id, enrollment_date FROM enrollment", conn, parse_dates = ['enrollment_date'])
        conn.close()
        enrollment = enrollment_db.append(enrollment_new)
        
    #drop and rename cols for Cognify reports
    utl_dict['inpatient'].drop(inpatient_drop, axis=1, inplace=True)
    utl_dict['inpatient'].rename(columns=inpatient_cols, inplace=True)

    utl_dict['er_adm'].drop(er_drop, axis=1, inplace=True)
    utl_dict['er_adm'].rename(columns=er_cols, inplace=True)

    utl_dict['er_non'].drop(er_drop, axis=1, inplace=True)
    utl_dict['er_non'].rename(columns=er_cols, inplace=True)

    cognify_faciliity_fix = {'Our Lady of Fatima Hospital': 'Fatima Hospital',
                            'The Miriam Hospital' : 'Miriam Hospital',
                            'Westerly Hospital ': 'Westerly Hospital',
                            'Roger Williams Hospital': 'Roger Williams Medical Center',
                            'Roger Williams Cancer Center': 'Roger Williams Medical Center',
                            'St. Elizabeth Manor': 'St. Elizabeth Home'}
    
    #replace some Cognify report facility quirks
    #can these be changed when being input?
    utl_dict['inpatient']['facility'].replace(cognify_faciliity_fix,
                                          inplace=True)
    utl_dict['er_adm']['facility'].replace(cognify_faciliity_fix,
                                          inplace=True)
    utl_dict['er_non']['facility'].replace(cognify_faciliity_fix,
                                          inplace=True)

    utl_dict['inpatient']['merge'] = (utl_dict['inpatient']['member_id'].astype(str) +
                                  utl_dict['inpatient']['admission_date'].astype(str) +
                                  utl_dict['inpatient']['facility'])

    utl_dict['er_adm']['merge'] = (utl_dict['er_adm']['member_id'].astype(str) +
                                  utl_dict['er_adm']['admission_date'].astype(str) +
                                  utl_dict['er_adm']['facility'])

    utl_dict['er_non']['merge'] = (utl_dict['er_non']['member_id'].astype(str) +
                                  utl_dict['er_non']['admission_date'].astype(str) +
                                  utl_dict['er_non']['facility'])

    #Cognify inpatetient does not indicate ER or not
    #check if inpatient stay is in the ER admited report
    utl_dict['inpatient']['er'] = np.where(utl_dict['inpatient']['merge'].isin(utl_dict['er_adm']['merge'].tolist()),
                                                       1, 0)

    #change some hospital names in the manual utl grids
    utl_hospital = {'Kent' : 'Kent Hospital',
                'kent': 'Kent Hospital',
                'RIH' : 'Rhode Island Hospital', 
                'Landmark' : 'Landmark Medical Center', 
                'Miriam' : 'Miriam Hospital',
                'RWMC' : 'Roger Williams Medical Center',
                'Butler' : 'Butler Hospital',
                'SCH' : 'South County Hospital',
                'Fatima' : 'Fatima Hospital',
                'FirstHealth*' : 'FirstHealth Moore Reginal Hospital',
                'East Side urgent care' : 'East Side Urgent Care',
                'Westerly': 'Westerly Hospital',
                'SCC *': 'Carolinas Hosptial System',
                'W&I' : 'Women & Infants Hospital'}

    utl_dict['ut_grid_inp']['Hospital'].replace(utl_hospital, inplace=True)

    utl_dict['ut_grid_er']['Hospital'].replace(utl_hospital, inplace=True)

    #converting the xls file to csv if giving us some ghost rows
    utl_dict['ut_grid_inp'].dropna(subset=['Member ID'], inplace=True)
    utl_dict['ut_grid_er'].dropna(subset=['Member ID'], inplace=True)

    utl_dict['ut_grid_inp']['Member ID'] = utl_dict['ut_grid_inp']['Member ID'].astype(int)
    utl_dict['ut_grid_er']['Member ID'] = utl_dict['ut_grid_er']['Member ID'].astype(int)

    utl_dict['ut_grid_inp']['merge'] = (utl_dict['ut_grid_inp']['Member ID'].astype(str) +
                                  utl_dict['ut_grid_inp']['Date of visit'].astype(str) +
                                  utl_dict['ut_grid_inp']['Hospital'])

    #utl_dict['ut_grid_inp']['merge'] = utl_dict['ut_grid_inp']['merge'].str.replace('.', '')

    utl_dict['ut_grid_er']['merge'] = (utl_dict['ut_grid_er']['Member ID'].astype(str) +
                                  utl_dict['ut_grid_er']['Date of visit'].astype(str) +
                                  utl_dict['ut_grid_er']['Hospital'])
    #utl_dict['ut_grid_er']['merge'] = utl_dict['ut_grid_inp']['merge'].str.replace('.', '')
                                  
    #check that all UR Grid visits are in Cognify correctly
    grid_vs_cognify = utl_dict['ut_grid_inp'][(-utl_dict['ut_grid_inp']['merge'].isin(utl_dict['inpatient']['merge'].tolist()) &
                        utl_dict['ut_grid_inp']['Hospital'].notnull())][['Member ID',
                                                                          'Ppt Name', 
                                                                          'Date of visit', 
                                                                          'Date of Discharge',
                                                                          'Hospital', 'merge']]
    
    if not grid_vs_cognify.empty:
        grid_vs_cognify.to_csv('ut_grid_cognify_diffs.csv', index=False)
        utl_dict['inpatient'][utl_dict['inpatient']['member_id'].isin(grid_vs_cognify['Member ID'])].to_csv('inp_for_diff.csv', index=False)
        raise AssertionError('All UR Grid Inpatient Visits are not in Cognify correctly.')

    #check that all UR Grid ER visits are either in the Cognify Inpatient Report
    #or in the ER Non Admit Report
    #this is where there are Cognify inpatient stays in the UT Grid ER Tab
    er_inp_check = utl_dict['inpatient'][(utl_dict['inpatient']['merge'].isin(utl_dict['ut_grid_er']['merge'].tolist()))]['merge']

    #this is all UT Grid ER Visits that are not in the Congify ER Only List
    missing_cog = utl_dict['ut_grid_er'][(-(utl_dict['ut_grid_er']['merge'].isin(utl_dict['er_non']['merge'].tolist())) &
                                      utl_dict['ut_grid_er']['Hospital'].notnull())][['Member ID', 'Ppt Name',
                                                                                       'Date of visit', 'Date of Discharge',
                                                                                       'Hospital', 'merge']]
    #this is any UR Grid ER Visits that is not in either Cognify report
    missing_ur_er_visits = missing_cog[-(missing_cog['merge'].isin(er_inp_check))]
    
    if not missing_ur_er_visits.empty:
        missing_ur_er_visits.to_csv('ut_grid_cognify_diffs_er.csv', index=False)
        raise AssertionError('All UR Grid ER Visits are not in Cognify correctly.')

    #drop cols
    utl_dict['ut_grid_inp'].drop(utl_drop_cols, axis=1, inplace=True)
    utl_dict['ut_grid_er'].drop(utl_drop_cols, axis=1, inplace=True)

    #rename cols
    utl_dict['ut_grid_inp'].rename(columns=utl_rename_dict, inplace=True)
    utl_dict['ut_grid_er'].rename(columns=utl_rename_dict, inplace=True)

    #replace weird excel blank values with np.nan
    utl_dict['ut_grid_inp']['days_MD'] = np.where(utl_dict['ut_grid_inp']['days_MD'].astype(float)>=1000, np.nan, utl_dict['ut_grid_inp']['days_MD'])
    utl_dict['ut_grid_inp']['days_RN'] = np.where(utl_dict['ut_grid_inp']['days_RN'].astype(float)>=1000, np.nan, utl_dict['ut_grid_inp']['days_RN'])

    utl_dict['ut_grid_er']['days_MD'] = np.where(utl_dict['ut_grid_er']['days_MD'].astype(float)>=1000, np.nan, utl_dict['ut_grid_er']['days_MD'])
    utl_dict['ut_grid_er']['days_RN'] = np.where(utl_dict['ut_grid_er']['days_RN'].astype(float)>=1000, np.nan, utl_dict['ut_grid_er']['days_RN'])

    #drop any scheduled visits, data is not tracked for these
    drop_sched = utl_dict['ut_grid_inp'][utl_dict['ut_grid_inp']['visit_type'] == 'Scheduled'].index.tolist()
    utl_dict['ut_grid_inp'].drop(drop_sched, inplace=True)

    utl_dict['ut_grid_inp'].drop('visit_type', axis=1, inplace=True)

    #create OBS column
    utl_dict['ut_grid_er']['visit_type'] = np.where(utl_dict['ut_grid_er']['visit_type'] == 'OBS', 1, 0)
    utl_dict['ut_grid_er'].rename(columns={'visit_type':'observation'}, inplace=True)

    inp_obs = utl_dict['ut_grid_er'][(utl_dict['ut_grid_er']['merge'].isin(er_inp_check)) & 
                       (utl_dict['ut_grid_er']['observation'] == 1)]['merge'].tolist()

    utl_dict['inpatient']['observation'] = np.where(utl_dict['inpatient']['merge'].isin(inp_obs), 1, 0)

    #removal of UR Grid ER Visits that are not ER Only but instead
    #are ER to inpatient
    utl_dict['ut_grid_er'] = utl_dict['ut_grid_er'][-utl_dict['ut_grid_er']['merge'].isin(er_inp_check)]

    #Seperate out inpatient Hospital and Inpatient Psych Visits
    #this is so days since last visit refers to a visit of a certain type

    hosp_list = ['Rhode Island Hospital', 'Landmark Medical Center', 
                'Kent Hospital', 'Roger Williams Hospital', 'Fatima Hospital',
                'Miriam Hospital', 'Westerly Hospital', 'Women & Infants Hospital']

    fill_admission_type_na = ['Acute Hospital' if facility in hosp_list else np.nan for facility in utl_dict['inpatient']['facility']]

    utl_dict['inpatient']['admission_type'] = np.where(utl_dict['inpatient']['admission_type'].isnull(),
                                                        fill_admission_type_na,
                                                        utl_dict['inpatient']['admission_type'])

    inpatient_hosp = utl_dict['inpatient'][utl_dict['inpatient'].admission_type.isin(['Acute Hospital',
                                                                                  np.nan])].copy().reset_index(drop=True)

    inpatient_pysch = utl_dict['inpatient'][utl_dict['inpatient'].admission_type.isin(['Psych Unit / Facility',
                                                                                  np.nan])].copy().reset_index(drop=True)

    #Create datetime Cols
    inpatient_hosp['admission_date'] = pd.to_datetime(inpatient_hosp['admission_date'])
    inpatient_hosp['discharge_date'] = pd.to_datetime(inpatient_hosp['discharge_date'])

    inpatient_pysch['admission_date'] = pd.to_datetime(inpatient_pysch['admission_date'])
    inpatient_pysch['discharge_date'] = pd.to_datetime(inpatient_pysch['discharge_date'])

    inpatient_hosp.drop_duplicates(inplace=True)
    inpatient_pysch.drop_duplicates(inplace=True)

    #Calculate Day Since Last Admit
    inpatient_hosp = discharge_admit_diff(inpatient_hosp, sql_table='inpatient', admit_type='Acute Hospital', update=update)
    inpatient_pysch = discharge_admit_diff(inpatient_pysch, sql_table='inpatient', admit_type='Psych Unit / Facility', update=update)

    #these are all weird copies that are checking the Admission Date against the same stays Discharge Date
    inpatient_hosp['days_since_last_admission'] = np.where(inpatient_hosp['days_since_last_admission'] <= 0, np.nan,
                                                           inpatient_hosp['days_since_last_admission'])

    inpatient_pysch['days_since_last_admission'] = np.where(inpatient_pysch['days_since_last_admission'] <= 0, np.nan,
                                                            inpatient_pysch['days_since_last_admission'])
       
    #put them back together
    inpatient = inpatient_hosp.append(inpatient_pysch)

    #merge inpatient Cognify report with the UR Grid Inpatient
    inpatient = inpatient.merge(utl_dict['ut_grid_inp'], on='merge', how='left').drop_duplicates()
  
    #merge with enrollment to create a column indicating
    #if the visit was within 6 months of enrollment

    inpatient = inpatient.merge(enrollment[['member_id','enrollment_date']])

    inpatient['enrollment_date'] = pd.to_datetime(inpatient['enrollment_date'])
    inpatient['admission_date'] = pd.to_datetime(inpatient['admission_date'])

    inpatient['w_six_months'] = np.where((inpatient['admission_date'].dt.to_period('M') -
                                           inpatient['enrollment_date'].dt.to_period('M')).apply(lambda x: x.freqstr[:-1]).replace('',
                                                                                                                                       np.nan).astype(float) <= 6,
                                          1, 0)

    #Create day of the week column
    inpatient['dow'] = inpatient['admission_date'].dt.weekday
    inpatient['dow'].replace({0:'Monday', 1:'Tuesday', 2:'Wednesday',
    3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}, inplace=True)
    
    #Code any Yes/No cols as 1/0
    code_y_n(inpatient)

    #Drop left over enrollment data and the merge column - we are done with them
    inpatient.drop(['enrollment_date', 'merge'], axis=1, inplace=True)

    #create datetime cols
    inpatient['admission_date'] = pd.to_datetime(inpatient['admission_date'])
    inpatient['discharge_date'] = pd.to_datetime(inpatient['discharge_date'])

    inpatient.drop_duplicates(subset = ['member_id', 'admission_date', 'facility'], inplace=True)

    for col in inpatient.columns:
        if inpatient[col].dtype == 'O':
            inpatient[col] = inpatient[col].str.title()

    #Merge Cognify ER Only and UR Grid ER reports
    er_only = utl_dict['er_non'].merge(utl_dict['ut_grid_er'], on='merge', how='left').drop_duplicates()

    er_only = er_only.merge(enrollment[['member_id','enrollment_date']])
    
    #merge with enrollment to create a column indicating
    #if the visit was within 6 months of enrollment
    er_only['enrollment_date'] = pd.to_datetime(er_only['enrollment_date'])
    er_only['admission_date'] = pd.to_datetime(er_only['admission_date'])

    er_only['w_six_months'] = np.where((er_only['admission_date'].dt.to_period('M') - 
                                    er_only['enrollment_date'].dt.to_period('M')).apply(lambda x: x.freqstr[:-1]).replace('',
                                                                                                                         np.nan).astype(float) <= 6,
                                          1, 0)
    
    #Create day of the week column
    er_only['dow'] = er_only['admission_date'].dt.weekday
    er_only['dow'].replace({0:'Monday', 1:'Tuesday', 2:'Wednesday',
    3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}, inplace=True)
    
    #Code any Yes/No cols as 1/0
    code_y_n(er_only)

    #Drop left over enrollment data and the merge column - we are done with that
    er_only.drop(['enrollment_date', 'merge'], axis=1, inplace=True)

    #Create datetime cols
    er_only['admission_date'] = pd.to_datetime(er_only['admission_date'])

    er_only.drop_duplicates(inplace=True)

    #find days between visits for participants
    er_only = discharge_admit_diff(er_only, sql_table='er_only', admit_diff=True, update=update)

    #these are all weird copies that are checking the Admission Date against the same stays Discharge Date
    er_only['days_since_last_admission'] = np.where(er_only['days_since_last_admission'] <= 0, np.nan,
                                                    er_only['days_since_last_admission'])

    for col in er_only.columns:
        if er_only[col].dtype == 'O':
            er_only[col] = er_only[col].str.title()


    er_only.drop_duplicates(subset = ['member_id', 'admission_date', 'facility'], inplace=True)

    #start on Nursing Facility table
    inpatient_snf = utl_dict['inpatient'][utl_dict['inpatient']['admission_type'].isin(['Nursing Home',
                                                                                    'Rehab Unit / Facility',
                                                                                    'End of Life'])].copy()
    #replace long admit reasons with the core reason custodial/respite/skilled/other
    cust = {x:'custodial' for x in ['custodial placement', 'custodial stay', 'Custodial placement',
                         'Custodial', 'resumed custodial care', 'custodial palcement',
                         'EOL', 'custodial stay for EOL care', 'end of life', 'Custodial',
                         'custodial stay for colonscopy prep', 'eol care', 'need for long term care']}

    res = {x:'respite' for x in ['respite stay', 're', 'emergent respite stay', 'Respite', 'emergent respite']}

    oth = {x:'other' for x in ['behavioral concerns/unable to be managed at home', 'functional decline']}

    skill = {x:'skilled' for x in ['skilled services', 'sk', 'skilled stay', 'Skilled', 'weakness/PT/OT',
                       'skileld services', 'skilled','rehab', 'Rehab and Nursing Services',
                       'need for skilled wound care and pain management', 'Wound care', 'rehab']}

    reason_dict = {**cust, **res, **oth, **skill}

    inpatient_snf['admit_reason'].replace(reason_dict, inplace=True)

    #calculate LOS
    inpatient_snf['los'] = np.where(inpatient_snf.discharge_date.isnull(), np.nan, inpatient_snf.los)

    #create datetime cols
    inpatient_snf['admission_date'] = pd.to_datetime(inpatient_snf['admission_date'])
    inpatient_snf['discharge_date'] = pd.to_datetime(inpatient_snf['discharge_date'])

    #merge with enrollment to create a column indicating
    #if the visit was within 6 months of enrollment
    inpatient_snf = inpatient_snf.merge(enrollment[['member_id',
                                                          'enrollment_date']])

    inpatient_snf['enrollment_date'] = pd.to_datetime(inpatient_snf['enrollment_date'])

    inpatient_snf['w_six_months'] = np.where((inpatient_snf['admission_date'].dt.to_period('M') - 
                                          inpatient_snf['enrollment_date'].dt.to_period('M')).apply(lambda x: x.freqstr[:-1]).replace('',
                                                                                                                                     np.nan).astype(float) <= 6,
                                          1, 0)

    inpatient_snf.drop(['enrollment_date', 'er', 'merge'], axis=1, inplace=True)

    inpatient_snf.drop_duplicates(inplace=True)

    inpatient_snf = discharge_admit_diff(inpatient_snf,  sql_table='inpatient_snf', update=update)

    #these are all weird copies that are checking the Admission Date against the same stays Discharge Date
    inpatient_snf['days_since_last_admission'] = np.where(inpatient_snf['days_since_last_admission'] <= 0, np.nan,
                                                          inpatient_snf['days_since_last_admission'])

    inpatient_snf.drop_duplicates(subset = ['member_id', 'admission_date', 'facility'], inplace=True)

    for col in inpatient_snf.columns:
        if inpatient_snf[col].dtype == 'O':
            inpatient_snf[col] = inpatient_snf[col].str.title()

    return inpatient, er_only, inpatient_snf
